In [345]:
import numpy as np
import pandas as pd
import csv, math

# Step 1
Import Given data into this notebook as variables to be used with programs.

The Coordinates, angles and distances are imported as arrays that will be called on throughout the program. 
The Calculated Coordinates file was created via an excel sheet used to create the initial values of the coordinates.

we have n 12 distance observations, and 19 angle observations. so n = 31 and u 16 unknowns (8 coordinates, 2 at eact)

In [509]:
#This code opens and reads the distances, angles, and calcualted coordinate files and stores them 
# As dictionaries that can be used to call specific values.
tcoords = []
cdict ={}
angles = []
d = []
ddict={}
with open('CalculatedCoordinates.csv', newline = '') as cd:
    coords = csv.reader(cd, delimiter=',')
    for cd in coords:
        tcoords.append((cd))
    tcoords = np.asarray(tcoords, dtype=float)
    for i in range(len(tcoords)):
        cdict[int(tcoords[i][0])]=[tcoords[i][1],tcoords[i][2]]


with open('angles3.txt', newline = '') as ag:
    angle = csv.reader(ag, delimiter =' ')
    for ag in angle:
        angles.append(ag)

    angles = np.asarray(angles, dtype=float)

    for i in range(len(angles)):
        angles[i][3] = 360-angles[i][3]


with open('distances3.txt', newline='') as D:
    distance = csv.reader(D, delimiter=' ')
    for D in distance:
        d.append(D)
    d = np.asarray(d, dtype=float)
    for i in range(len(d)):
        ddict[int(d[i][0])]=[d[i][1],d[i][2]]


# Step 2

Begin computing the coordinates of each point using the differential equations to solve 

\begin{gather}
Y_c=

In [522]:

#Create initial matrices 
#A is design matrix
# Lo is measured observations
# Xu is the unkowns
# w is the misclosures

A = np.zeros((len(angles)+len(d),len(tcoords)*2))
Lo = np.zeros((len(angles)+len(d),1))
w = np.zeros((len(angles)+len(d),1))

    #inverse equation
def tinverse(Xj,Yj,Xi,Yi):
    dx = Xj-Xi
    dy = Yj-Yi
    #print(Xj,Xi,Yj,Yi)
    x = math.atan((dx)/(dy))
    
    x = math.degrees(x)
    
    if dx > 0 and dy > 0:
        x=x
    elif dx > 0 and dy < 0:
        x=x+180
    elif dx < 0 and dy <0:
        x=x+180
    else:
        x=x+360
    return x

    def approx_length(xi,yi,xj,yj):
        x = math.sqrt(((xj-xi)**2)+((yj-yi)**2))
        return x



#creating functions to calculate the partial derivative (pd) of the distance equations
def pd(i,j,ij):
    x = (i-j)/ij
    return x

def kij(Lij,Xi,Xj,Yi,Yj):
    IJ = (((Xj-Xi)**2)+((Yj-Yi))**2)**(1/2)
    Kij = Lij-IJ
    return Kij


#iterate through the distance observations, calculating each partial derivative and storign them into the A matrix
def Angles_Distance_Partials(angles,d,cdict):
    A = np.zeros((len(angles)+len(d),len(tcoords)*2))
    Lo = np.zeros((len(angles)+len(d),1))
    w = np.zeros((len(angles)+len(d),1))

    # putting the distance and angular observations into the L matrix

    for i in range(len(d)):
        Lo[i] = d[i][1]
    for i in range(len(angles)):
        j = i+12
        Lo[j][0] = angles[i][3]
        j=0
        



    for i in range(len(d)):
        line = int(d[i][0])

        begin = int(str(line)[0])
        end = int(str(line)[1])
        
        llength = ddict[line][0]


        Xi = cdict[int(str(line)[0])][0]
        Yi = cdict[int(str(line)[0])][1]
        Xj = cdict[int(str(line)[1])][0]
        Yj = cdict[int(str(line)[1])][1]
        
        pdXi = pd(Xi,Xj,llength)
        pdYi = pd(Yi,Yj,llength)
        pdXj = pd(Xj,Xi,llength)
        pdYj = pd(Yj,Yi,llength)
        
        Kij = kij(llength,Xi,Xj,Yi,Yj)
        

        
        Olength = approx_length(Xi,Yi,Xj,Yj)
        wlength = Olength - llength
        
        A[i][begin-1]= pdXi
        A[i][end-1]= pdXj
        A[i][begin+7]= pdYi
        A[i][end+7]= pdYj
        w[i] = wlength
    # iterate through the angles matrix and calculate each partial derivative for the angular equations

    for i in range(len(angles)):
        
        
        #isolating the coordinate of each station of the angle
        from_ = angles[i][0]
        at_ = angles[i][1]
        to_ = angles[i][2]
        
        

        
        
        #storing the x and y coordinates into there own variables
        # a is at       f is from       t is to
        fx = cdict[from_][0]
        fy = cdict[from_][1]
        
        ax = cdict[at_][0]
        ay = cdict[at_][1]
        
        tx = cdict[to_][0]
        ty = cdict[to_][1]
        
        # calculating the two azimuths of the angle
        
        af = tinverse(fromx,fromy,atx,aty)
        at = tinverse(tox,toy,atx,aty)
        
        # each partial derivative
        # a is at       f is from       t is to
        # xp and yp means the partials
        
        axp = (((fy-ay)/(af**2))-((ty-ay)/(at**2))) 
        ayp = (((ax-fx)/(af**2))-((ax-fx)/(at**2)))
        fxp = ((ay-fy)/(af**2))
        fyp = ((fx-ax)/(af**2)) 
        txp = ((fy-ay)/(at**2))
        typ = ((ax-fx)/(at**2))
        
        def AngleP(fx,fy,ax,ay,tx,ty,af,at):
            
            
            p = 206265
            Kbif = p*((ay-fy)/(af**2))   +   p*((fx-ax)/(af**2))    +  p*(((fy-ay)/(af**2))-((ty-ay)/(at**2))) +   p*(((ax-fx)/(af**2))-((ax-fx)/(at**2))) + p*((fy-ay)/(at**2)) + p*((ax-fx)/(at**2))
            return Kbif
        
        # interior angle BIF is IF - IB     At-To  -  At-From
        
        theta = tinverse(fx,fy,ax,ay) - tinverse(tx,ty,ax,ay)
        if theta < 0:
            theta += 360
        w_interior_angle = theta - angles[i][3]
        
        partial_angle = AngleP(fx,fy,ax,ay,tx,ty,af,at)
        

        from_ = int(from_)
        at_ = int(at_)
        to_ = int(to_)
        
        c = int(len(d))
        A[i+c][from_-1] = fxp
        A[i+c][at_-1] = axp
        A[i+c][to_-1] = txp
        A[i+c][from_+7] = fyp
        A[i+c][at_+7] = ayp
        A[i+c][to_+7] = fyp
        w[i+c] = w_interior_angle

    return A, w, Lo

def rearrangeA(A):
    a = A
    permutation = [0,2,4,6,8,10,12,14,1,3,5,7,9,11,13,15]
    idx = np.empty_like(permutation)
    idx[permutation] = np.arange(len(permutation))
    x = a[:, idx]  # return a rearranged copy
    return x

A, w, Lo = Angles_Distance_Partials(angles,d,cdict)
A = rearrangeA(A)

# Part 3, Establishing the covariance matrix

Now that the design matrix A, vector of observations L and vector of misclosures w, we now need to find our weight matrix P, which is defined by being the inverse matrix of our covariance.